In [1]:
#import dependencies
import pandas as pd
import numpy as np

#load file
heroes_data = pd.read_csv("Resources/HeroesOfPymoli_Purchase_Data.csv")
heroes_data_df = pd.DataFrame(heroes_data)
heroes_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [2]:
# Display total players
total_players = heroes_data.loc[:, ["Gender", "SN", "Age"]]
total_players = total_players.drop_duplicates()
num_of_players = total_players.count()[0]
pd.DataFrame({"Total Players": [num_of_players]})

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
#Display Purchasing Analysis with unique items, average price, number of purchases and total revenue
unique_number = len(heroes_data_df['Item ID'].unique())
avg_price = round(float(heroes_data_df['Price'].mean()),2)
num_purchases = len(heroes_data_df['Price'])
total_revenue = float(heroes_data_df['Price'].sum())

pd.DataFrame({"Number of Unique Items": [unique_number], "Average Price": "$" + str(avg_price), "Number of Purchases": [num_purchases], "Total Revenue": "$" + str(total_revenue)})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

In [4]:
#Display percentage and count of Female and Male, etc.
gender_count = heroes_data_df["Gender"].value_counts(0)
percent_player = heroes_data_df["Gender"].value_counts(1)

gender_count_df = pd.DataFrame(gender_count)
percent_player_df = round(pd.DataFrame(percent_player) * 100, 2)


gender_demographics_df = gender_count_df.merge(percent_player_df, left_index = True, right_index = True)
gender_demographics_df.columns = ["Total Count", "Percentage of Players"]
gender_demographics_df

#format the columns
gender_demographics_df["Percentage of Players"] = gender_demographics_df["Percentage of Players"].map("{}%".format)
gender_demographics_df

,Total Count,Percentage of Players
Male,652,83.59%
Female,113,14.49%
Other / Non-Disclosed,15,1.92%


## Purchasing Analysis (Gender)

In [5]:
#Display the purchase count, average purchase price, total purchase value and average total purchase per person by gender.
gender_purchase = heroes_data_df.groupby('Gender')

purchase_count = round(gender_purchase['Purchase ID'].count(),0)
avg_purchase = round(gender_purchase['Price'].mean(), 2)
total_pur_val = round(gender_purchase['Price'].sum(), 2)
avg_total_purchase_per_person = round(total_pur_val/ gender_count, 2)


purchase_analysis_gender = pd.DataFrame([purchase_count, avg_purchase, total_pur_val, avg_total_purchase_per_person])
purchase_analysis = purchase_analysis_gender.T
purchase_analysis.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Total Purchase Per Person"]
purchase_analysis

#format columns
purchase_analysis["Average Purchase Price"] = purchase_analysis["Average Purchase Price"].map("${:.2f}".format) 
purchase_analysis["Total Purchase Value"] = purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
purchase_analysis["Average Total Purchase Per Person"] = purchase_analysis["Average Total Purchase Per Person"].map("${:.2f}".format)

purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Gender,,,,
Female,113.0,$3.20,$361.94,$3.20
Male,652.0,$3.02,$1967.64,$3.02
Other / Non-Disclosed,15.0,$3.35,$50.19,$3.35


## Age Demographics

In [6]:
#create bins for ages
agebins = [0, 9.9, 14.9, 19.9, 24.90, 29.9, 34.9, 39.9, 100]
group_bins = ["10 and Under", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40 and Over"]

#categorize existing players with age bins
total_players["Age Ranges"] = pd.cut(total_players["Age"], agebins, labels=group_bins)

#calculate the numbers and percentages by Age Group
age_dem_tots = total_players ["Age Ranges"].value_counts()
age_dem_perc = age_dem_tots / num_of_players
age_dem = pd.DataFrame({"Total Count": age_dem_tots, "Percentage of Players": age_dem_perc})
age_dem ["Percentage of Players"] = age_dem["Percentage of Players"].map("{:,.2%}".format)

age_dem = age_dem.sort_index()
age_dem

,Total Count,Percentage of Players
10 and Under,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40 and Over,12,2.08%


## Purchasing Analysis (Age)

In [7]:
heroes_data["Age Ranges"] = pd.cut(heroes_data["Age"], agebins, labels=group_bins)

#obtain purchase count, avg. purchase price with basic calculations
age_purchase_totes = heroes_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_avg = heroes_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_counts = heroes_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

norm_purchase_totes = age_purchase_totes / age_dem["Total Count"]

age_data = pd.DataFrame({"Purchase Count": age_counts, "Average Purchase Price": age_avg, "Total Purchase Value": age_purchase_totes, "Normalized Totals": norm_purchase_totes})

# Organize the data to look cleaner
age_data["Average Purchase Price"] = age_data["Average Purchase Price"].map("${:,.2f}".format)
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map("${:,.2f}".format)
age_data["Normalized Totals"] = age_data["Normalized Totals"].map("${:,.2f}".format)

age_data



,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Ranges,,,,
10 and Under,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40 and Over,13,$2.94,$38.24,$3.19


## Top Spenders

In [8]:
SN_totes = heroes_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
SN_avg = heroes_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
SN_purchases = heroes_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

SN_data = pd.DataFrame({"Total Purchase Value": SN_totes, "Average Purchase Price": SN_avg, "Purchase Count": SN_purchases})
SN_data_table = SN_data.sort_values("Total Purchase Value", ascending=False)

# Organize the data to look cleaner
SN_data_table["Total Purchase Value"] = SN_data_table["Total Purchase Value"].map("${:,.2f}".format)
SN_data_table["Average Purchase Price"] = SN_data_table["Average Purchase Price"].map("${:,.2f}".format)

SN_data_table.head(5)

,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,$18.96,$3.79,5
Idastidru52,$15.45,$3.86,4
Chamjask73,$13.83,$4.61,3
Iral74,$13.62,$3.40,4
Iskadarya95,$13.10,$4.37,3


## Most Popular Items

In [13]:
popular_item = heroes_data.loc[:,["Item ID", "Item Name", "Price"]]

purchase_item_totes = popular_item.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
purchase_item_avg = popular_item.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_count = popular_item.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")

                
popular_items = pd.DataFrame({"Total Purchase Value": purchase_item_totes, "Item Price": purchase_item_avg, "Purchase Count": item_count})
popular_item_sorted = popular_items.sort_values("Purchase Count", ascending=False)

# Organize the data to look cleaner
popular_item_sorted["Item Price"] = popular_item_sorted["Item Price"].map("${:,.2f}".format)
popular_item_sorted["Purchase Count"] = popular_item_sorted["Purchase Count"].map("${:,.2f}".format)
popular_item_sorted["Total Purchase Value"] = popular_item_sorted["Total Purchase Value"].map("${:,.2f}".format)
popular_data = popular_item_sorted.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

popular_data.head(5)                                         
                                         

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,$13.00,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",$12.00,$4.23,$50.76
145,Fiery Glass Crusader,$9.00,$4.58,$41.22
132,Persuasion,$9.00,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",$9.00,$3.53,$31.77


## Most Profitable Items